In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/oral_cancer_prediction"
!ls -la "$PROJECT_DIR"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 24
drwx------ 2 root root 4096 Oct 23 21:22 configs
drwx------ 2 root root 4096 Oct 23 21:17 data
drwx------ 2 root root 4096 Nov 15 03:02 models
drwx------ 2 root root 4096 Oct 24 17:39 notebooks
drwx------ 2 root root 4096 Nov 15 03:02 results
drwx------ 2 root root 4096 Nov 15 03:10 src


In [ ]:
%%writefile /content/drive/MyDrive/oral_cancer_prediction/src/train.py
import yaml
import os
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix
)

# YAML config
CONFIG_PATH = "/content/drive/MyDrive/oral_cancer_prediction/configs/baseline.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

# Extract config parts
data_cfg = config["data"]
model_cfg = config["model"]
output_cfg = config["output"]
project_cfg = config["project"]
tuning_cfg = config.get("tuning", {"enabled": False})

# Loading dataset
df = pd.read_csv(data_cfg["path"])
X = df[data_cfg["features"]].copy()
y = df[data_cfg["target"]].copy()

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=project_cfg["test_size"],
    random_state=project_cfg["random_state"],
    stratify=y_encoded,
)

# Preprocessing- OneHotEncoder
preprocess = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), data_cfg["features"])]
)

#  Model setup
rf_params = model_cfg["parameters"]
rf = RandomForestClassifier(**rf_params)

pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", rf)
])

if tuning_cfg["enabled"]:
    print(" Running RandomizedSearchCV for hyperparameter tuning...")
    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions={
            f"model__{k}": v for k, v in tuning_cfg["param_distributions"].items()
        },
        n_iter=tuning_cfg["n_iter"],
        cv=tuning_cfg["cv_folds"],
        scoring=tuning_cfg["scoring"],
        n_jobs=-1,
        random_state=tuning_cfg["random_state"],
        verbose=2
    )
    search.fit(X_train, y_train)
    pipeline = search.best_estimator_
    print("\n Best Parameters Found:")
    for k, v in search.best_params_.items():
        print(f"{k}: {v}")
else:
    pipeline.fit(X_train, y_train)
#Train
#pipeline.fit(X_train, y_train)

#Evaluating
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

acc  = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec  = recall_score(y_test, y_pred)
f1   = f1_score(y_test, y_pred)
auc  = roc_auc_score(y_test, y_proba)

report = classification_report(y_test, y_pred, target_names=le.classes_)
cm = confusion_matrix(y_test, y_pred)

print("==== RESULTS ====")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC-AUC  : {auc:.4f}")
print("\nClassification Report:\n", report)
print("Confusion Matrix:\n", cm)

# Save model & metrics
os.makedirs(os.path.dirname(output_cfg["model_path"]), exist_ok=True)
os.makedirs(os.path.dirname(output_cfg["metrics_path"]), exist_ok=True)

joblib.dump(pipeline, output_cfg["model_path"])

with open(output_cfg["metrics_path"], "w") as f:
    f.write(
        f"Accuracy:  {acc:.6f}\n"
        f"Precision: {prec:.6f}\n"
        f"Recall:    {rec:.6f}\n"
        f"F1:        {f1:.6f}\n"
        f"ROC-AUC:   {auc:.6f}\n\n"
        f"Confusion Matrix:\n{cm}\n\n"
        f"{report}\n"
    )

print(f"\n Model saved to: {output_cfg['model_path']}")
print(f" Metrics saved to: {output_cfg['metrics_path']}")


Overwriting /content/drive/MyDrive/oral_cancer_prediction/src/train.py


In [ ]:
# Run training from the script (single source of truth)
!python /content/drive/MyDrive/oral_cancer_prediction/src/train.py

 Running RandomizedSearchCV for hyperparameter tuning...
Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] END model__class_weight=None, model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=6, model__min_samples_split=2, model__n_estimators=500; total time=  37.6s
[CV] END model__class_weight=None, model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=6, model__min_samples_split=2, model__n_estimators=500; total time=  39.5s
[CV] END model__class_weight=None, model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=6, model__min_samples_split=2, model__n_estimators=500; total time=  36.4s
[CV] END model__class_weight=None, model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=6, model__min_samples_split=2, model__n_estimators=500; total time=  38.6s
[CV] END model__class_weight=None, model__max_depth=None, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=5, model__n_estimator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/oral_cancer_prediction"
!ls -la "$PROJECT_DIR"

Mounted at /content/drive
total 24
drwx------ 2 root root 4096 Oct 23 21:22 configs
drwx------ 2 root root 4096 Oct 23 21:17 data
drwx------ 2 root root 4096 Oct 23 21:22 models
drwx------ 2 root root 4096 Oct 24 17:39 notebooks
drwx------ 2 root root 4096 Oct 23 21:22 results
drwx------ 2 root root 4096 Oct 23 21:22 src


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.5 MB/s eta 0:00:00


In [ ]:
%%writefile /content/drive/MyDrive/oral_cancer_prediction/src/catboost_train_syn_v2.py
import yaml
import os
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix, roc_curve
)

from catboost import CatBoostClassifier

# ============================================================
# Load YAML
# ============================================================
CONFIG_PATH = "/content/drive/MyDrive/oral_cancer_prediction/configs/syn_cb_config_v2.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)

data_cfg = config["data"]
model_cfg = config["model"]
output_cfg = config["output"]
project_cfg = config["project"]
tuning_cfg = config.get("tuning", {"enabled": False})

# ============================================================
# Load dataset
# ============================================================
df = pd.read_csv(data_cfg["path"])

cat_cols = data_cfg["features"]["categorical"]
num_cols = data_cfg["features"]["numeric"]

missing = [c for c in (cat_cols + num_cols) if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns: {missing}")

X = df[cat_cols + num_cols].copy()
y = df[data_cfg["target"]].copy()

le = LabelEncoder()
y_encoded = le.fit_transform(y)

# ============================================================
# Train-test split
# ============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y_encoded,
    test_size=project_cfg["test_size"],
    random_state=project_cfg["random_state"],
    stratify=y_encoded,
)

cat_indexes = [X.columns.get_loc(c) for c in cat_cols]

# ============================================================
# Base model
# ============================================================
params = model_cfg["parameters"]
model = CatBoostClassifier(**params)

# ============================================================
# Hyperparameter Tuning
# ============================================================
if tuning_cfg["enabled"]:
    print("\n Running RandomizedSearchCV...")

    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=tuning_cfg["param_distributions"],
        n_iter=tuning_cfg["n_iter"],
        cv=tuning_cfg["cv_folds"],
        scoring=tuning_cfg["scoring"],
        random_state=tuning_cfg["random_state"],
        verbose=2,
        n_jobs=-1,
        refit=True
    )

    search.fit(X_train, y_train, cat_features=cat_indexes)

    print("\n Best Parameters Found:")
    for k, v in search.best_params_.items():
        print(f"{k}: {v}")

    # CV results
    cv_df = pd.DataFrame(search.cv_results_)
    print("\n TOP 10 CV RESULTS")
    print(
        cv_df[["mean_test_score", "std_test_score"]]
        .sort_values("mean_test_score", ascending=False)
        .head(10)
    )

    # Save CV results
    cv_df.to_csv(output_cfg["cv_results_csv"], index=False)
    print("\n CV results saved to:", output_cfg["cv_results_csv"])

    model = search.best_estimator_

else:
    model.fit(X_train, y_train, cat_features=cat_indexes)

# ============================================================
# Training performance
# ============================================================
y_train_pred = model.predict(X_train)
y_train_proba = model.predict_proba(X_train)[:, 1]

train_acc = accuracy_score(y_train, y_train_pred)
train_auc = roc_auc_score(y_train, y_train_proba)

print("\n===== TRAINING PERFORMANCE =====")
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Train AUC:      {train_auc:.4f}")

# ============================================================
# Test performance
# ============================================================
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)

cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=le.classes_)

print("\n===== TEST RESULTS =====")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"ROC-AUC  : {auc:.4f}")
print("Confusion Matrix:\n", cm)
print(report)

# ============================================================
# Threshold Optimization
# ============================================================
fpr, tpr, thr = roc_curve(y_test, y_proba)
best_thr = thr[(tpr - fpr).argmax()]
print(f"\nOptimal Threshold: {best_thr:.4f}")

y_opt = (y_proba >= best_thr).astype(int)

opt_acc = accuracy_score(y_test, y_opt)
opt_prec = precision_score(y_test, y_opt)
opt_rec = recall_score(y_test, y_opt)
opt_f1 = f1_score(y_test, y_opt)

print("\n===== OPTIMIZED THRESHOLD RESULTS =====")
print(f"Opt Accuracy : {opt_acc:.4f}")
print(f"Opt Precision: {opt_prec:.4f}")
print(f"Opt Recall   : {opt_rec:.4f}")
print(f"Opt F1-score : {opt_f1:.4f}")

# ============================================================
# Save model + metrics
# ============================================================
os.makedirs(os.path.dirname(output_cfg["model_path"]), exist_ok=True)
os.makedirs(os.path.dirname(output_cfg["metrics_path"]), exist_ok=True)

joblib.dump(model, output_cfg["model_path"])

with open(output_cfg["metrics_path"], "w") as f:
    f.write("DEFAULT THRESHOLD (0.5) METRICS\n")
    f.write(f"Accuracy:  {acc:.6f}\n")
    f.write(f"Precision: {prec:.6f}\n")
    f.write(f"Recall:    {rec:.6f}\n")
    f.write(f"F1-score:  {f1:.6f}\n")
    f.write(f"AUC:       {auc:.6f}\n\n")

    f.write("Confusion Matrix (Default Threshold):\n")
    f.write(f"{cm}\n\n")
    f.write("Classification Report (Default Threshold):\n")
    f.write(report + "\n\n")

    f.write(" OPTIMIZED THRESHOLD METRICS \n")
    f.write(f"Optimal Threshold: {best_thr:.6f}\n")
    f.write(f"Opt Accuracy:      {opt_acc:.6f}\n")
    f.write(f"Opt Precision:     {opt_prec:.6f}\n")
    f.write(f"Opt Recall:        {opt_rec:.6f}\n")
    f.write(f"Opt F1-score:      {opt_f1:.6f}\n\n")

    opt_cm = confusion_matrix(y_test, y_opt)
    opt_report = classification_report(y_test, y_opt, target_names=le.classes_)

    f.write("Confusion Matrix (Optimized Threshold):\n")
    f.write(f"{opt_cm}\n\n")

    f.write("Classification Report (Optimized Threshold):\n")
    f.write(opt_report)

print("\n Model saved to:", output_cfg["model_path"])
print(" Metrics saved to:", output_cfg["metrics_path"])


Overwriting /content/drive/MyDrive/oral_cancer_prediction/src/catboost_train_syn_v2.py


In [ ]:
!python /content/drive/MyDrive/oral_cancer_prediction/src/catboost_train_syn_v2.py


 Running RandomizedSearchCV...
Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV] END bagging_temperature=0.5, bootstrap_type=Bayesian, border_count=64, depth=6, iterations=800, l2_leaf_reg=5, learning_rate=0.01, min_data_in_leaf=1, random_strength=5; total time=   5.5s
[CV] END bagging_temperature=0.5, bootstrap_type=Bayesian, border_count=64, depth=6, iterations=800, l2_leaf_reg=5, learning_rate=0.01, min_data_in_leaf=1, random_strength=5; total time=   5.7s
[CV] END bagging_temperature=0.5, bootstrap_type=Bayesian, border_count=64, depth=6, iterations=800, l2_leaf_reg=5, learning_rate=0.01, min_data_in_leaf=1, random_strength=5; total time=   4.8s
[CV] END bagging_temperature=0.5, bootstrap_type=Bayesian, border_count=64, depth=6, iterations=800, l2_leaf_reg=5, learning_rate=0.01, min_data_in_leaf=1, random_strength=5; total time=   4.7s
[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=64, depth=5, iterations=400, l2_leaf_reg=5, learning_rate=0.